In [1]:
include("../src/Julia.jl")

objc[50350]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[50350]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[50350]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[50350]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


pcgWorker (generic function with 1 method)

In [106]:
gname = "../../graphs/randdense_iu_1000_1/"

"../../graphs/randdense_iu_1000_1/"

In [107]:
a = readFromFile(gname * "graph.mtx");

In [108]:
tree = []
str = []
data = []

for i in 1:3
    curTree = readFromFile(gname * "tree$(i).mtx");
    rest = a - curTree
    curStr = compStretches(curTree,rest).nzval
    
    push!(tree, curTree)
    push!(str, curStr)
    push!(data, (sum(curStr)/2, mean(curStr)))
end

In [109]:
# check these values when editing the description
n = a.n;
iu = true
p = 0.25;
wmax = 1000;

In [110]:
function strDist{Tv,Ti}(tr::SparseMatrixCSC{Tv,Ti}, a::SparseMatrixCSC{Tv,Ti})
    
    res = Array{ASCIIString,1}(0)
    
    str = compStretches(tr, a-tr).nzval;
    
    vmax = maximum(str)
    inter = (0,1)
    while inter[2] < vmax
        cnt = 0
        for i in 1:length(str)
            if inter[1] <= str[i] && str[i] < inter[2]
                cnt = cnt + 1
            end
        end
        # don't count edges twice
        cnt = ceil(Int64,cnt / 2)
        
        push!(res, "[$(inter[1]),$(inter[2])) - $(cnt)")
        
        inter = (inter[2], inter[2] * 2)
    end
    
    push!(res, "")
    
    return res;
    
end

strDist (generic function with 1 method)

In [111]:
# Return the sorted eigenvalues of lap(a) * lap(tr)^-1
function getEigs{Tv,Ti}(tr::SparseMatrixCSC{Tv,Ti}, a::SparseMatrixCSC{Tv,Ti})
    
    la = lap(a);
    ts = treeSolver(tr);
    
    function f(b)
        return lap(a) * ts(b)
    end
    
    fOp = SqLinOp(true,1.0,a.n,f)
    
    eigVals = eigs(fOp,nev=a.n-1)[1]
    
    eigsString = ""
    for i in length(eigVals):-1:1
        eigsString = eigsString * string(round(eigVals[i],4)) * " "
    end
    
    return eigsString
    
end

getEigs (generic function with 1 method)

In [112]:
output = []
push!(output, "A random graph with $(n) vertices and $(ceil(Int64,nnz(a)/2)) edges.")
push!(output, "")
push!(output, "The graph is generated in the following way:")
push!(output, "    - for each vertex, select n / $(1/p) random neighbors")
push!(output, "    - get rid of duplicate edges")
if iu
    push!(output, "    - weights are reals sampled uniformly from 1/[1..$(wmax)]")
else
    push!(output, "    - weights are integers sampled uniformly from [1..$(wmax)]")
end
push!(output, "")
push!(output, "There are three trees in the folder.")
push!(output, "")

for i in 1:3
    push!(output, "Tree$(i) - total stretch = $(data[i][1]), ave stretch = $(data[i][2])")
    @time append!(output, strDist(tree[i],a))
    push!(output, "Eigenvalues of lap(a) * lap(tree$(i))^-1")
    @time push!(output, getEigs(tree[i],a))
    push!(output, "")
    push!(output, "")
end

desc = open(gname * "description2.txt", "w")
for i in 1:length(output)
    println(desc, output[i])
end
close(desc)

  0.044405 seconds (10.11 k allocations: 19.047 MB)
  9.166755 seconds (13.50 M allocations: 3.729 GB, 8.71% gc time)
  0.033853 seconds (276 allocations: 18.585 MB, 11.44% gc time)
  9.328259 seconds (13.51 M allocations: 3.729 GB, 6.34% gc time)
  0.030464 seconds (292 allocations: 18.585 MB, 10.33% gc time)
  8.748713 seconds (13.48 M allocations: 3.729 GB, 6.98% gc time)
